# Modelos

In [212]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC


In [197]:
df = pd.read_pickle('df_transformado.pkl')
df_test = pd.read_pickle('df_test_transformado.pkl')


In [194]:
df.head()

,__Gender_Female,__Gender_Male,id,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad
0,0.0,1.0,0,24,1.699998,81.66995,1,1,2,3,1,0,2.763573,1,0.0,0.976473,1,3
1,1.0,0.0,1,18,1.56,57.0,1,1,2,3,2,0,2.0,1,1.0,1.0,0,1
2,1.0,0.0,2,18,1.71146,50.165754,1,1,2,1,1,0,1.910378,1,1.0,1.673584,0,0
3,1.0,0.0,3,21,1.71073,131.274851,1,1,3,3,1,0,1.674061,1,1.5,0.780199,1,6
4,0.0,1.0,4,32,1.914186,93.798055,1,1,3,2,1,0,1.979848,1,2.0,0.931721,1,3


In [198]:
y = df['NObeyesdad']
id = df['id']
X = df.drop(['NObeyesdad','id'], axis=1)

y = y.to_numpy()
X = X.to_numpy()

id_test = df_test['id']
X_test = df_test.drop(['id'],axis=1).to_numpy()

In [179]:
nobsedy_invertio = {3: 'Overweight_Level_II', 1: 'Normal_Weight', 0: 'Insufficient_Weight',
                     6: 'Obesity_Type_III', 5: 'Obesity_Type_II', 2: 'Overweight_Level_I', 4: 'Obesity_Type_I'}


## KNN

In [217]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = KNeighborsClassifier(n_neighbors=10)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))



fold 0> accuracy =  0.8540462427745664
fold 1> accuracy =  0.8554913294797688
fold 2> accuracy =  0.848747591522158
fold 3> accuracy =  0.853082851637765
fold 4> accuracy =  0.8453757225433526
fold 5> accuracy =  0.8458574181117534
fold 6> accuracy =  0.8598265895953757
fold 7> accuracy =  0.846820809248555
fold 8> accuracy =  0.8563855421686747
fold 9> accuracy =  0.8587951807228915
Accuracy Mean: 0.852, Standard Deviation: 0.005


In [181]:
knnmodel = KNeighborsClassifier()

## Floresta de Árvore de Decisão

In [182]:
# k-fold cross validation
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model = RandomForestClassifier(n_estimators=300)
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9084778420038536
fold 1> accuracy =  0.8916184971098265
fold 2> accuracy =  0.8858381502890174
fold 3> accuracy =  0.8930635838150289
fold 4> accuracy =  0.8945086705202312
fold 5> accuracy =  0.896917148362235
fold 6> accuracy =  0.901252408477842
fold 7> accuracy =  0.888728323699422
fold 8> accuracy =  0.9012048192771084
fold 9> accuracy =  0.896867469879518
Accuracy Mean: 0.896, Standard Deviation: 0.006


In [211]:
rf_model = RandomForestClassifier(n_estimators=300)
rf_model.fit(X,y)
y_pred = rf_model.predict(X_test)


In [ ]:
df_random_forest = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_random_forest['NObeyesdad']  = df_random_forest['NObeyesdad'].map(nobsedy_invertio)

df_random_forest.to_csv('submtions/randomforest.csv', index=False, sep=',')


## Gradient Boost

O algoritmo foi testado de três formas, removendo CH2O, TUE e FAF, teve perdas pequenas no treinamento. Ao usar n_estimator=300, o treinamento teve bons resultados, todavia a acuracia caiu no teste.

In [203]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	model =  GradientBoostingClassifier()
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))

fold 0> accuracy =  0.9051059730250481
fold 1> accuracy =  0.9036608863198459
fold 2> accuracy =  0.8930635838150289
fold 3> accuracy =  0.896917148362235
fold 4> accuracy =  0.905587668593449
fold 5> accuracy =  0.9046242774566474
fold 6> accuracy =  0.9104046242774566
fold 7> accuracy =  0.8921001926782274
fold 8> accuracy =  0.9074698795180723
fold 9> accuracy =  0.9050602409638554
Accuracy Mean: 0.902, Standard Deviation: 0.006


In [201]:
gb_model = GradientBoostingClassifier()
gb_model.fit(X,y)
y_pred = gb_model.predict(X_test)

In [202]:
df_gradient_boosting = pd.DataFrame({'id': id_test, 'NObeyesdad': y_pred})
df_gradient_boosting['NObeyesdad']  = df_gradient_boosting['NObeyesdad'].map(nobsedy_invertio)

df_gradient_boosting.to_csv('submtions/gradient_boosting.csv', index=False, sep=',')

## Voting Classifier

In [ ]:
scores = list()
kfold = KFold(10, shuffle=True, random_state=4)

for k, (train, test) in enumerate(kfold.split(X, y)):
	# get data
	train_X, test_X = X[train], X[test]
	train_y, test_y = y[train], y[test]
	# fit model
	clf2 = RandomForestClassifier(n_estimators=300, random_state=1)
	clf1 = GradientBoostingClassifier()
	clf3 = GradientBoostingClassifier(n_estimators=300)
	model = VotingClassifier( estimators=[('gdb', clf1), ('rf', clf2), ('gdb300', clf3)],  voting='hard')
	model.fit(train_X, train_y)
	# evaluate model
	ypred = model.predict(test_X)
	acc = accuracy_score(test_y, ypred)
	# store score
	scores.append(acc)
	print(f'fold {k}> accuracy = ', acc)

# summarize model performance
mean_s, std_s = np.mean(scores), np.std(scores)
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean_s, std_s))